In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [2]:
sc=SparkContext()      

In [30]:
path = "eventLogging/local-1588781816130"
spark = SQLContext(sc)

df=sqlConf.read.json(path)

df.createOrReplaceTempView("Events")


In [27]:
# The Number of events so far
print(f'The number of events So far = {df.count()}')

The number of events So far = 81


In [16]:
df.select("App Name").dropna().collect()

[Row(App Name='PySparkShell')]

In [36]:
dist_events=df.groupBy('Event').count()
print(f'The number of distinct events So far = {dist_events.count()}')

The number of distinct events So far = 11


In [44]:
event_list=[]
event_timestamp=dict()
for event in dist_events.collect():
    
    event_dict=event.asDict()
    
    event_list.append(event_dict.['Event'])
    
    event_timestamp[event_dict.['Event']]=event_dict.['Event']
     

In [64]:
open_event=[]

for event in event_list:
    if event[-5:]=='Start':
        open_event.append(event[:-5])
        
    elif event[-3:]=='End':
        open_event.remove(event[:-3])
        
    

In [59]:
print(f'The events that are still open is {open_event}')

The events that are still open is ['SparkListenerLog', 'SparkListenerApplication']


In [113]:

event_time_dict=dict()
for event in df.collect():
    
    event_d=event.asDict()
    try:
        start_time = event_d['Task Info']['Launch Time']
        end_time = event_d['Task Info']['Finish Time']
        
        if end_time > 0:            
            try:
                event_time_dict[event_d['Event']]+=','+ str(end_time-start_time)
            except:
                event_time_dict[event_d['Event']]=str(end_time-start_time)
    except:
        pass
            

In [140]:
for task, time in event_time_dict.items():
    
 s_time=time.split(',')

 avg=sum(list(map(int,s_time)))/len(s_time)
 print(f' The task {task} has the average time {avg}')
 


 The task SparkListenerTaskEnd has the average time 238.3846153846154


In [144]:
for task, time in event_time_dict.items():
    
 s_time=time.split(',')

 l_time=list(map(int,s_time))
    
 long_job=[]
 
for e_time in l_time:
    if e_time > avg:
        long_job.append((task,e_time))
        
        
        
print(f'The long job list is {long_job}')


The long job list is [('SparkListenerTaskEnd', 616), ('SparkListenerTaskEnd', 620), ('SparkListenerTaskEnd', 621), ('SparkListenerTaskEnd', 621), ('SparkListenerTaskEnd', 623), ('SparkListenerTaskEnd', 621), ('SparkListenerTaskEnd', 621), ('SparkListenerTaskEnd', 660), ('SparkListenerTaskEnd', 486)]


In [18]:
from pyspark.sql.types import StructField
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType

In [26]:
schemaString = "name age city address postcode"

fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)

In [27]:
schema

StructType(List(StructField(name,StringType,true),StructField(age,StringType,true),StructField(city,StringType,true),StructField(address,StringType,true),StructField(postcode,StringType,true)))

In [32]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [34]:
from pyspark.sql import Row

sc = spark.sparkContext

# Load a text file and convert each line to a Row.
lines = sc.textFile("people.txt")
parts = lines.map(lambda l: l.split(","))
people = parts.map(lambda p: Row(name=p[0], age=int(p[1])))



In [39]:
parts.collect()

[['Michael', ' 29'], ['Andy', ' 30'], ['Justin', ' 19']]

In [40]:
# Infer the schema, and register the DataFrame as a table.
schemaPeople = spark.createDataFrame(people)
schemaPeople.createOrReplaceTempView("people")

# SQL can be run over DataFrames that have been registered as a table.
teenagers = spark.sql("SELECT name FROM people WHERE age >= 13 AND age <= 19")

# The results of SQL queries are Dataframe objects.
# rdd returns the content as an :class:`pyspark.RDD` of :class:`Row`.
teenNames = teenagers.rdd.map(lambda p: "Name: " + p.name).collect()
for name in teenNames:
    print(name)
# Name: Justin

Name: Justin


In [43]:
teenNames

['Name: Justin']